# CNN Pipeline v8

## Environment settings

### Libraries

In [1]:
import tensorflow as tf
import numpy as np
import os
import random
import pandas as pd
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from PIL import Image, ImageFilter, ImageEnhance
from numpy import asarray
from sklearn.utils import shuffle
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from IPython.display import clear_output
import math

tfk = tf.keras
tfkl = tf.keras.layers
print(tf.__version__)

2.7.0


### Random seed

In [2]:
# Random seed for reproducibility
seed = 42

random.seed(seed)
os.environ['PYTHONHASHSEED'] = str(seed)
np.random.seed(seed)
tf.random.set_seed(seed)
tf.compat.v1.set_random_seed(seed)

## Data Loader

In [3]:
# google drive plug-in
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [4]:
# directory
%cd /gdrive/MyDrive/notebooks/an2dl_homeworks/training_validation_testing/mirko

/gdrive/MyDrive/notebooks/an2dl_homeworks/training_validation_testing/mirko


In [9]:
# paths
root_path = '../../training_validation_testing/mirko/'#'/kaggle/input/leaves-2/leaves_train_val/'#
training_dir = os.path.join(root_path, 'training')
validation_dir = os.path.join(root_path, 'validation')

## Data Pre-Processing

### Generators

#### Training set

In [27]:
# ------------------------------------------------------------------------------
# FIXED TRAINING ---------------------------------------------------------------
# ------------------------------------------------------------------------------

# MEAN CENTER SHIFT
z_score_mean = [[[51.14068, 68.74184, 40.975986]]]

# STD NORMALIZATION
z_score_std = [[[81.23509, 104.63732, 66.81536]]]

# ------------------------------------------------------------------------------
# ------------------------------------------------------------------------------

# constructor
train_data_gen = ImageDataGenerator(
    # data augmentation
    rotation_range=30,
    height_shift_range=50,
    width_shift_range=50,
    zoom_range=0.3,
    horizontal_flip=True,
    vertical_flip=True,
    brightness_range=[0.5,1.5],
    fill_mode='reflect',

    # z-score
    featurewise_center=False, # mean center shift
    featurewise_std_normalization=False # std normalization
)

# ------------------------------------------------------------------------------

# generator
train_gen = train_data_gen.flow_from_directory(
    directory=training_dir,
    target_size=(256,256),
    color_mode='rgb',
    classes=None,
    class_mode='categorical',
    batch_size=32,
    shuffle=True,
    seed=seed
)

# ------------------------------------------------------------------------------

# custom
def train_gen_z_score():
  # yielding batch's samples at run time when called from the generator
  for (x, y) in train_gen:
    x = (x - z_score_mean) / z_score_std
    yield (x, y)

#### New Training set

In [ ]:
# ------------------------------------------------------------------------------
# NEW TRAINING -----------------------------------------------------------------
# ------------------------------------------------------------------------------

# constructor
train_data_gen = ImageDataGenerator(
    # data augmentation
    rotation_range=30,
    height_shift_range=50,
    width_shift_range=50,
    zoom_range=0.3,
    horizontal_flip=True,
    vertical_flip=True,
    brightness_range=[0.5,1.5],
    fill_mode='reflect',

    # z-score
    featurewise_center=False, # mean center shift
    featurewise_std_normalization=False # std normalization
)

# ------------------------------------------------------------------------------

# generator
train_gen = train_data_gen.flow_from_directory(
    directory= root_path + 'training',
    target_size=(256,256),
    color_mode='rgb',
    classes=None,
    class_mode='categorical',
    batch_size=32,
    shuffle=True,
    seed=seed
)

# ------------------------------------------------------------------------------

# callable generator method
def CallableGenerator():
  # yielding batch's samples at run time when called from the generator
  for (x,y) in train_gen:
    yield (x,y)

# TENSOR-BATCHED DATASET (from generator)
batched_dataset = tf.data.Dataset.from_generator(
    # above method for run time
    CallableGenerator,

    # tensor fields types
    output_types=(tf.float32, tf.float32)
) 

# TENSOR-UNBATCHED DATASET (from batched-dataset)
mean = tf.Variable(
    # RGB dimension        
    np.zeros((3), dtype=np.float32)
)
mean_2 = tf.Variable(
    # RGB dimension        
    np.zeros((3), dtype=np.float32)
)
std = tf.Variable(
    # RGB dimension        
    np.zeros((3), dtype=np.float32)
)

# visualization variables
i = 0 # counter
tot_epoch = math.ceil(train_gen.samples / 32) # total epoch

# ONE EPOCH LOADING
for (x, _) in iter(batched_dataset):
  # update visualization
  clear_output(wait=True)
  i += 1
  print("Epoch (1/1) - Batch : " + str(i) + "/" + str(tot_epoch))

  # incremental mean
  mean.assign_add(
      tf.math.divide(
        tf.math.reduce_sum(x, axis=[0, 1, 2]),
        256 * 256 * train_gen.samples
      )
  )

  # incremental mean squared
  mean_2.assign_add(
      tf.math.divide(
        tf.math.reduce_sum(tf.math.square(x), axis=[0, 1, 2]),
        256 * 256 * train_gen.samples
      )
  )
    
  # end condition, otherwise unlimited
  if i > tot_epoch:
      break

# standard deviation
std = tf.math.sqrt(
    tf.math.subtract(mean_2, mean)
)

# ------------------------------------------------------------------------------

# reshape
z_score_mean = tf.reshape(mean, (1, 1, 3))
z_score_std = tf.reshape(std, (1, 1, 3))

# visualization
print('training set mean :', z_score_mean)
print('training set std :', z_score_std)

# ------------------------------------------------------------------------------

# generator
train_gen = train_data_gen.flow_from_directory(
    directory=training_dir,
    target_size=(256,256),
    color_mode='rgb',
    classes=None,
    class_mode='categorical',
    batch_size=32,
    shuffle=True,
    seed=seed
)

# custom
def train_gen_z_score():
  # yielding batch's samples at run time when called from the generator
  for (x, y) in train_gen:
    x = (x - z_score_mean) / z_score_std
    yield (x, y)

#### Validation set

In [30]:
# VALIDATION -------------------------------------------------------------------

# constructor
valid_data_gen = ImageDataGenerator(
    # z-score
    featurewise_center=False, # mean center shift
    featurewise_std_normalization=False # std normalization
)

# generator
valid_gen = valid_data_gen.flow_from_directory(
    directory=validation_dir,
    target_size=(256,256),
    color_mode='rgb',
    classes=None,
    class_mode='categorical',
    batch_size=32,
    shuffle=True,
    seed=seed
)

def valid_gen_z_score():
  # yielding batch's samples at run time when called from the generator
  while True:
    for (x, y) in train_gen:
      x = (x - z_score_mean) / z_score_std
      yield (x, y)

Found 3547 images belonging to 14 classes.


## Model Design

In [31]:
# labels
labels = ['Apple','Blueberry','Cherry','Corn','Grape','Orange','Peach','Pepper','Potato','Raspberry','Soybean','Squash','Strawberry','Tomato']

In [32]:
# Model hyperparameters
input_shape = (256, 256, 3)
epochs = 100
batch_size = 32
n_classes = len(labels)
weight_decay = 1e-5
model_name = "cnn_v1"

In [33]:
# Model skeleton
def mirknet(input_shape):
    # (0) input
    input_layer = tfkl.Input(shape=input_shape, name='Input')

    # (1) 
    conv1 = tfkl.Conv2D(
        filters=16,
        kernel_size=(3, 3),
        strides = (1, 1),
        padding = 'same',
        kernel_initializer = tfk.initializers.GlorotUniform(seed),
        kernel_regularizer=tf.keras.regularizers.l2(weight_decay)
    )(input_layer)
    drop1 = tfkl.Dropout(0.5, seed=seed)(conv1)
    relu1 = tf.keras.layers.ReLU()(drop1)
    pool1 = tfkl.MaxPooling2D(
        pool_size = (2, 2)
    )(relu1)

    # (2)
    conv2 = tfkl.Conv2D(
        filters=32,
        kernel_size=(3, 3),
        strides = (1, 1),
        padding = 'same',
        kernel_initializer = tfk.initializers.GlorotUniform(seed)
    )(pool1)
    drop2 = tfkl.Dropout(0.6, seed=seed)(conv2)
    relu2 = tf.keras.layers.ReLU()(drop2)
    pool2 = tfkl.MaxPooling2D(
        pool_size = (2, 2)
    )(relu2)

    # (3)
    conv3 = tfkl.Conv2D(
        filters=64,
        kernel_size=(3, 3),
        strides = (1, 1),
        padding = 'same',
        activation = 'relu',
        kernel_initializer = tfk.initializers.GlorotUniform(seed)
    )(pool2)
    drop3 = tfkl.Dropout(0.7, seed=seed)(conv3)
    relu3 = tf.keras.layers.ReLU()(drop3)
    pool3 = tfkl.MaxPooling2D(
        pool_size = (2, 2)
    )(relu3)

    # (4) 
    conv4 = tfkl.Conv2D(
        filters=128,
        kernel_size=(3, 3),
        strides = (1, 1),
        padding = 'same',
        activation = 'relu',
        kernel_initializer = tfk.initializers.GlorotUniform(seed)
    )(pool3)
    drop4 = tfkl.Dropout(0.8, seed=seed)(conv4)
    relu4 = tf.keras.layers.ReLU()(drop4)
    pool4 = tfkl.MaxPooling2D(
        pool_size = (2, 2)
    )(relu4)

    # (5)
    conv5 = tfkl.Conv2D(
        filters=256,
        kernel_size=(3, 3),
        strides = (1, 1),
        padding = 'same',
        activation = 'relu',
        kernel_initializer = tfk.initializers.GlorotUniform(seed)
    )(pool4)
    drop5 = tfkl.Dropout(0.9, seed=seed)(conv5)
    relu5 = tf.keras.layers.ReLU()(drop5)
    pool5 = tfkl.MaxPooling2D(
        pool_size = (2, 2)
    )(relu5)

    # (5) 
    conv6 = tfkl.Conv2D(
        filters=512,
        kernel_size=(3, 3),
        strides = (1, 1),
        padding = 'same',
        activation = 'relu',
        kernel_initializer = tfk.initializers.GlorotUniform(seed)
    )(pool5)
    batch_norm6 = tfkl.BatchNormalization()(conv6)
    relu6 = tf.keras.layers.ReLU()(batch_norm6)
    gap6 = tfkl.GlobalAveragePooling2D()(relu6)

    # (6)
    flattening_layer = tfkl.Flatten(name='Flatten')(gap6)

    flattening_layer = tfkl.Dropout(0.5, seed=seed)(flattening_layer)
    classifier_layer = tfkl.Dense(units=512,
                                  name='Classifier_1',
                                  kernel_initializer=tfk.initializers.GlorotUniform(seed),
                                  kernel_regularizer=tf.keras.regularizers.l2(weight_decay),
                                  activation='relu')(flattening_layer)

    flattening_layer = tfkl.Dropout(0.6, seed=seed)(classifier_layer)
    classifier_layer = tfkl.Dense(units=256,
                                  name='Classifier_2',
                                  kernel_initializer=tfk.initializers.GlorotUniform(seed),
                                  activation='relu')(flattening_layer)

    flattening_layer = tfkl.Dropout(0.7, seed=seed)(classifier_layer)
    classifier_layer = tfkl.Dense(units=128,
                                  name='Classifier_3',
                                  kernel_initializer=tfk.initializers.GlorotUniform(seed),
                                  activation='relu')(flattening_layer)

    flattening_layer = tfkl.Dropout(0.8, seed=seed)(classifier_layer)
    classifier_layer = tfkl.Dense(units=64,
                                  name='Classifier_4',
                                  kernel_initializer=tfk.initializers.GlorotUniform(seed),
                                  activation='relu')(flattening_layer)

    flattening_layer = tfkl.Dropout(0.9, seed=seed)(classifier_layer)
    classifier_layer = tfkl.Dense(units=32,
                                  name='Classifier_5',
                                  kernel_initializer=tfk.initializers.GlorotUniform(seed),
                                  activation='relu')(flattening_layer)
    batch_norm = tfkl.BatchNormalization()(classifier_layer)

    output_layer = tfkl.Dense(units=n_classes, activation='softmax', 
                              kernel_initializer=tfk.initializers.GlorotUniform(seed), 
                              name='Output')(batch_norm)

    # Model connection
    model = tfk.Model(inputs=input_layer, outputs=output_layer, name='model')

    # Model Compilation
    model.compile(loss=tfk.losses.CategoricalCrossentropy(), optimizer=tfk.optimizers.Adam(), metrics='accuracy')

    # Return the model
    return model

In [34]:
# Build model
model = mirknet(input_shape)
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Input (InputLayer)          [(None, 256, 256, 3)]     0         
                                                                 
 conv2d_6 (Conv2D)           (None, 256, 256, 16)      448       
                                                                 
 dropout_10 (Dropout)        (None, 256, 256, 16)      0         
                                                                 
 re_lu_6 (ReLU)              (None, 256, 256, 16)      0         
                                                                 
 max_pooling2d_5 (MaxPooling  (None, 128, 128, 16)     0         
 2D)                                                             
                                                                 
 conv2d_7 (Conv2D)           (None, 128, 128, 32)      4640      
                                                             

In [35]:
# Utility function to create folders and callbacks for training
from datetime import datetime

def create_folders_and_callbacks(model_name):

  exps_dir = os.path.join('/kaggle/working/models')#root_path+'/new_model_gap')
  if not os.path.exists(exps_dir):
      os.makedirs(exps_dir)

  now = datetime.now().strftime('%b%d_%H-%M-%S')

  exp_dir = os.path.join(exps_dir, model_name + '_' + str(now))
  if not os.path.exists(exp_dir):
      os.makedirs(exp_dir)
      
  callbacks = []

  # Model checkpoint
  # ----------------
  ckpt_dir = os.path.join(exp_dir, 'ckpts')
  if not os.path.exists(ckpt_dir):
      os.makedirs(ckpt_dir)

  ckpt_callback = tf.keras.callbacks.ModelCheckpoint(filepath=os.path.join(ckpt_dir, 'cp.ckpt'), 
                                                     save_weights_only=False, # True to save only weights
                                                     save_best_only=False) # True to save only the best epoch 
  callbacks.append(ckpt_callback)

  # Early Stopping
  # --------------
  es_callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
  callbacks.append(es_callback)

  return callbacks

In [36]:
# Create folders and callbacks and fit
callbacks = create_folders_and_callbacks(model_name='CNN_full')

# Train the model
history = model.fit(
    x = train_gen_z_score(),
    batch_size = batch_size,
    epochs = epochs,
    validation_data = valid_gen_z_score(),
    callbacks=[callbacks],
    steps_per_epoch = train_gen.samples // batch_size
).history

Epoch 1/100
  7/332 [..............................] - ETA: 21:34 - loss: 3.1618 - accuracy: 0.0670

KeyboardInterrupt: ignored

In [ ]:
# Save best epoch model
model.save("models/CNN_full_best_" + str(datetime.now().strftime('%b%d_%H-%M-%S')))